# Assignment 3
## Matthew Wirasakti
## Problem 1

In [165]:
def separate_by_class(dataset):
    separated = dict()
    for i in range(len(dataset)):
        vector = dataset[i]
        class_value = vector[-1]
        if (class_value not in separated):
            separated[class_value]=list()
        separated[class_value].append(vector)
    return separated

In [166]:
# Test separating data by class
dataset=[[3.393533211,2.331273381,0],
         [3.110073483,1.781539638,0],
         [1.343808831,3.368360954,0],
         [3.582294042,4.67917911,0],
         [2.280362439,2.866990263,0],
         [7.423436942,4.696522875,1],
         [5.745051997,3.533989803,1],
         [9.172168622,2.511101045,1],
         [7.792783481,3.424088941,1],
         [7.939820817,0.791637231,1]]

separated = separate_by_class(dataset)

for label in separated:
    print(label)
    for row in separated[label]:
        print(row)

0
[3.393533211, 2.331273381, 0]
[3.110073483, 1.781539638, 0]
[1.343808831, 3.368360954, 0]
[3.582294042, 4.67917911, 0]
[2.280362439, 2.866990263, 0]
1
[7.423436942, 4.696522875, 1]
[5.745051997, 3.533989803, 1]
[9.172168622, 2.511101045, 1]
[7.792783481, 3.424088941, 1]
[7.939820817, 0.791637231, 1]


In [167]:
# Calculate the mean of a list of numbers
def mean(numbers):
    return sum(numbers) / float(len(numbers))

In [168]:
from math import sqrt
# Calculate the standard deviation of a list of numbers
def stdev(numbers):
    avg = mean(numbers)
    variance = sum([(x-avg)**2 for x in numbers]) / float(len(numbers) - 1)
    return sqrt(variance)

In [169]:
def summarize_dataset(dataset):
    summaries = [(mean(feature), stdev(feature), len(feature)) for feature in zip(*dataset)]
    del(summaries[-1]) # remove stats for class label
    return summaries

In [170]:
#Let's test our step 2
summary = summarize_dataset(dataset)
print(summary)

[(5.178333386499999, 2.7665845055177263, 10), (2.9984683241, 1.218556343617447, 10)]


In [171]:
def summarize_by_class(dataset):
    separated = separate_by_class(dataset)
    summaries = dict()
    for class_value, rows in separated.items():
        summaries[class_value] = summarize_dataset(rows)
    return summaries

In [172]:
#Let's test our step 3
summary = summarize_by_class(dataset)
for label in summary:
    print(label)
    for row in summary[label]:
        print(row)

0
(2.7420144012, 0.9265683289298018, 5)
(3.0054686692, 1.1073295894898725, 5)
1
(7.6146523718, 1.2344321550313704, 5)
(2.9914679790000003, 1.4541931384601618, 5)


In [173]:
from math import pi
from math import exp

def calculate_probability(x, mean, stdev):
    exponent = exp(-((x - mean)**2 / (2 * stdev**2)))
    return(1 / (sqrt(2 * pi) * stdev)) * exponent

In [174]:
print(calculate_probability(1.0,1.0,1.0))
print(calculate_probability(2.0,1.0,1.0))
print(calculate_probability(0.0,1.0,1.0))

0.3989422804014327
0.24197072451914337
0.24197072451914337


In [175]:
def calculate_class_probabilities(summaries, row):
    total_rows = sum([summaries[label][0][2] for label in summaries])
    probabilities = dict()
    for class_value, class_summaries in summaries.items():
        probabilities[class_value] = summaries[class_value][0][2] / float(total_rows)
        for i in range(len(class_summaries)):
            mean, stdev, count = class_summaries[i]
            probabilities[class_value] *= calculate_probability(row[i], mean, stdev)
    return probabilities

In [176]:
# testing step 5
summaries = summarize_by_class(dataset)

#testing point
test = dataset[0]
# first training example

probabilities = calculate_class_probabilities(summaries, test)
print(probabilities)

{0: 0.05032427673372076, 1: 0.00011557718379945765}


In [177]:
# Naive Bayes On The Iris Dataset
from csv import reader
from random import seed
from random import randrange
from math import sqrt
from math import exp
from math import pi
from sklearn.metrics import classification_report

In [178]:
# Load a CSV file
def load_csv(filename):
    dataset = list()
    with open(filename, 'r') as file:
        csv_reader = reader(file)
        for row in csv_reader:
            if not row:
                continue
            dataset.append(row)
    return dataset

# Convert string column to float
def str_column_to_float(dataset, column):
    for row in dataset:
        row[column] = float(row[column].strip())
        
# Convert string column to int        
def str_column_to_int(dataset, column):
    class_values = [row[column] for row in dataset]
    unique = set(class_values)
    lookup = dict()
    for i, value in enumerate(unique):
        lookup[value] = i
    for row in dataset:
        row[column] = lookup[row[column]]
    return lookup

# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / n_folds)
    for _ in range(n_folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split

# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
    folds = cross_validation_split(dataset, n_folds)
    scores = list()
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set,[])
        test_set = list()
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None
        predicted = algorithm(train_set, test_set, *args)
        actual = [row[-1] for row in fold]
        accuracy = accuracy_metric(actual, predicted)
        scores.append(accuracy)
        print(classification_report(actual, predicted, zero_division = 0))
    return scores

In [179]:
# Predict the class for a given row
def predict(summaries, row):
    probabilities = calculate_class_probabilities(summaries, row)
    best_label, best_prob = None, -1
    for class_value, probability in probabilities.items():
        if best_label is None or probability > best_prob:
            best_prob = probability
            best_label = class_value
    return best_label

In [180]:
# Naive Bayes Algorithm
def naive_bayes(train, test):
    summarize = summarize_by_class(train)
    predictions = list()
    for row in test:
        output = predict(summarize, row)
        predictions.append(output)
    return(predictions)

In [181]:
# Test Naive Bayes on Iris Dataset
seed(1)
filename = 'iris.csv'
dataset = load_csv(filename)
for i in range(len(dataset[0]) - 1):
    str_column_to_float(dataset, i)
# convert class column to integers
str_column_to_int(dataset, len(dataset[0]) - 1)

# evaluate algorithm
n_folds = 5
scores = evaluate_algorithm(dataset, naive_bayes, n_folds)
print('Scores: %s' %scores)
print('Mean Accuracy: %.3f%%' %(sum(scores)/float(len(scores))))

              precision    recall  f1-score   support

           0       0.80      1.00      0.89         8
           1       1.00      1.00      1.00        11
           2       1.00      0.82      0.90        11

    accuracy                           0.93        30
   macro avg       0.93      0.94      0.93        30
weighted avg       0.95      0.93      0.93        30

              precision    recall  f1-score   support

           0       1.00      0.91      0.95        11
           1       1.00      1.00      1.00        10
           2       0.90      1.00      0.95         9

    accuracy                           0.97        30
   macro avg       0.97      0.97      0.97        30
weighted avg       0.97      0.97      0.97        30

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      1.00      1.00         6
           2       1.00      1.00      1.00        14

    accuracy        

## Problem 2

In [182]:
# naive bayes on text dataset
import random
seed(1)
datasetText = list()
sample = list()
for _ in range (1000):
    sample = [random.randint(0, 3) for _ in range(10)]
    datasetText.append(sample)

for i in range(len(datasetText)):
    vocab = datasetText[i]
    if (vocab[2] >= 1 & vocab[4] >= 1 & vocab[8] >= 1 & vocab[9] == 1):
        vocab.append(1)
    else:
        vocab.append(0)

print(datasetText)
print()
n_folds = 5
scores = evaluate_algorithm(datasetText, naive_bayes, n_folds)
print('Scores: %s' %scores)
print('Mean Accuracy: %.3f%%' %(sum(scores)/float(len(scores))))

[[1, 0, 2, 0, 3, 3, 3, 3, 1, 0, 0], [3, 0, 3, 3, 0, 3, 2, 1, 0, 2, 0], [0, 0, 0, 0, 3, 1, 3, 0, 1, 3, 0], [3, 1, 2, 1, 1, 3, 2, 0, 3, 0, 0], [1, 2, 0, 2, 3, 1, 2, 2, 3, 3, 0], [0, 3, 1, 3, 3, 1, 2, 2, 0, 3, 0], [0, 1, 3, 2, 3, 0, 3, 0, 2, 3, 0], [1, 1, 1, 0, 1, 1, 3, 2, 2, 3, 0], [2, 0, 3, 1, 1, 3, 0, 3, 2, 1, 0], [3, 3, 2, 3, 2, 0, 2, 3, 0, 1, 0], [1, 1, 0, 2, 0, 0, 0, 0, 3, 0, 0], [2, 1, 2, 0, 1, 2, 2, 0, 1, 1, 1], [2, 1, 2, 2, 3, 2, 3, 3, 0, 0, 0], [2, 3, 2, 3, 1, 2, 0, 2, 1, 3, 1], [0, 1, 0, 3, 1, 0, 1, 3, 3, 1, 0], [3, 1, 0, 3, 2, 3, 0, 2, 1, 1, 0], [0, 2, 0, 0, 2, 2, 1, 3, 2, 1, 0], [0, 0, 1, 3, 1, 0, 3, 1, 2, 0, 0], [1, 3, 1, 3, 0, 3, 2, 3, 0, 2, 0], [3, 2, 0, 1, 1, 2, 1, 2, 3, 1, 0], [2, 0, 3, 2, 3, 1, 0, 0, 0, 1, 0], [1, 1, 1, 2, 2, 2, 2, 2, 2, 0, 0], [2, 1, 3, 1, 0, 2, 0, 3, 0, 3, 0], [1, 1, 2, 0, 3, 0, 1, 0, 2, 2, 0], [2, 0, 3, 2, 0, 0, 2, 0, 0, 0, 0], [3, 0, 0, 1, 1, 3, 1, 0, 3, 1, 0], [1, 1, 0, 3, 3, 2, 2, 3, 2, 0, 0], [1, 2, 0, 0, 0, 2, 2, 3, 3, 2, 0], [3, 0, 0, 2, 3, 0, 

              precision    recall  f1-score   support

           0       0.90      1.00      0.95       177
           1       1.00      0.13      0.23        23

    accuracy                           0.90       200
   macro avg       0.95      0.57      0.59       200
weighted avg       0.91      0.90      0.86       200

              precision    recall  f1-score   support

           0       0.92      1.00      0.96       184
           1       1.00      0.06      0.12        16

    accuracy                           0.93       200
   macro avg       0.96      0.53      0.54       200
weighted avg       0.93      0.93      0.89       200

              precision    recall  f1-score   support

           0       0.90      0.97      0.94       179
           1       0.29      0.10      0.14        21

    accuracy                           0.88       200
   macro avg       0.59      0.53      0.54       200
weighted avg       0.84      0.88      0.85       200

              preci

## Problem 2 Report
The problem we faced was to develop a code that, after training with a given set of data, could predict whether an SMS or email was spam or ham. This required developing or downloading a dataset that would be split into training and testing data, and then calculations would be made to determine specific probabilities of words.

Relevant Papers:
1. An evaluation of Naive Bayes variants in content-based learning for spam filtering
    
    Author: Seewald, Alexander K.
    
    Summary: The paper details on research to compare two methods of Naive Bayes that both have additional features to a currently used method called SpamBayes that has no additional features and remains to the traditional Naive Bayes algorithm.
    

2. Spam detection on social networks using cost-sensitive feature selection and ensemble-based regularized deep neural networks
    
    Authors: Barushka, Aliaksandr & Hajek, Petr
    
    Summary: The article explains the need for spam filtering systems. Going into detail on how the occurrence and complexity of spam has increased as social networks expand. The necessity to deal with spam has also increased due to the various issues spam can cause, such as, security issues, increase in distrust of social networks, and an overall problem of time in dealing with spam.

For this assignment I developed my own dataset to simplify the code and function for the Naive Bayes algorithm. The dataset developed is actually similar to a bag of words dataset. 1000 vectors are created, each with 10 data points that are assigned a random number from 0 to 3, inclusive. Each data point represents a "word" and the random number assigned details the number of times that word appears in that vector, or in the case of this assignment, the "message". Depending on some predetermined criteria, a 1 or 0 is appended to the end of each vector, a 1 labeling it as spam, and a 0 as ham. Then the dataset is put through the algorithm and tested on various metrics.

See cell above for experimentation and results with metrics.